# Importing Python Packages

In [50]:
import openpyxl
import xlrd
import os
import glob

from pandas import DataFrame as df
from scipy.stats import trim_mean, kurtosis
from scipy.stats.mstats import mode, gmean, hmean

In [51]:
# import python packages

# import nltk (natural language tool kit)
import nltk
# stopwords, FreqDist, word_tokenize
from nltk.corpus import stopwords
from nltk import FreqDist, word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import requests
from lxml import html
import csv
import pandas as pd

from os import path

import matplotlib.pyplot as plt
%matplotlib inline


#regular expression
import re
#csv
import csv
# Excel-like format
import pandas as pd
#data visualization
import matplotlib.pyplot as plt
% matplotlib inline

import numpy as np

import statsmodels.api as sm

import statsmodels.formula.api as smf

# Importing Data

## Opening the Correct Directory

In [52]:
os.chdir('C:\\Users\\millsj.ADMIN\\Desktop\\GENBA 894\\2018_DataChallenges_Teradata\\Events\\')

In [82]:
# filenames
excel_names = ["2013-2017 Bike Events.xlsx"]

# read them in
excels = [pd.ExcelFile(name) for name in excel_names]

# turn them into dataframes
frames = [x.parse(x.sheet_names[0],index_col=None) for x in excels]

# delete the first row for all frames except the first
# i.e. remove the header row -- assumes it's the first
frames[1:] = [df[1:] for df in frames[1:]]

# concatenate them..
events = pd.concat(frames)

# Check headers
events.head(3)

,Fiscal Year,Event ID,Security Category Name,Public Event Name,Event Created Date,Event Date,Event Goal($),Active Registrations,Inactive Registrations,Total Fees Paid,...,Total Not From Participant($),Total Team Gifts($),Total Event Gifts($),Total Offline Unconfirmed Gifts($),Street Address,City,State,Zip Code,Internal Event Name,Previous Event ID
0,2016,27178,OHA Bike Events,Bike MS: Buckeye Breakaway 2016,2015-10-08 10:19:00,2016-08-06 15:26:00,1600000,1371,66,38492.0,...,752017.31,7558.62,15610.0,18285.0,3581 Center Rd.,Brunswick,OH,44212.0,OHA 2016 Bike MS: Buckeye Breakaway,25890
1,2017,28918,CTN Bike Events,Bike MS: Windsor 2017,2016-11-16 11:31:00,2017-06-04 11:38:00,234500,358,0,12716.5,...,132312.77,4841.00,296.0,2063.5,5 Waterside Crossing,Windsor,CT,6095.0,CTN 2017 Bike MS: Windsor,27498
2,2017,28219,WAS Bike Events,Bike MS: Hope to a Cure 2017,2016-08-15 17:55:00,2017-05-20 10:54:00,70000,83,0,3527.5,...,49565.75,958.00,1400.0,1155.0,Girdwood Place and Hightower Road,Girdwood,AK,99587.0,WAS 2017 Bike MS: Hope to a Cure,27173


In [54]:
# convert string to integers
#biketeams['Team Captain Accept Email'] = (biketeams['Team Captain Accept Email'] == 'TRUE').astype(int)

In [55]:
# make sure the conversion worked
#biketeams.head(3)

# Looking for Data Anomalies and Cleaning Addresses

In [56]:
# List of all headers
# events['Fiscal Year','Event ID','Security Category Name','Public Event Name','Event Created Date','Event Date','Event Goal($)','Active Registrations','Inactive Registrations','Total Fees Paid','Total of All Confirmed Gifts($)','Total Online Gifts($)','Teams','Captains','Average Team Size','Team pages with custom URL','Emails Sent','Personal pages with custom URL','Total Offline Confirmed Gifts($)','Self Donors','Total From Participant($)','Non-self Donors','Total Not From Participant($)','Total Team Gifts($)','Total Event Gifts($)','Total Offline Unconfirmed Gifts($)','Street Address','City','State','Zip Code','Internal Event Name','Previous Event ID']

In [83]:
# counting zero values
print((events[['Fiscal Year','Event ID','Security Category Name','Public Event Name','Event Created Date','Event Date','Event Goal($)','Active Registrations','Inactive Registrations','Total Fees Paid','Total of All Confirmed Gifts($)','Total Online Gifts($)','Teams','Captains','Average Team Size','Team pages with custom URL','Emails Sent','Personal pages with custom URL','Total Offline Confirmed Gifts($)','Self Donors','Total From Participant($)','Non-self Donors','Total Not From Participant($)','Total Team Gifts($)','Total Event Gifts($)','Total Offline Unconfirmed Gifts($)','Street Address','City','State','Zip Code','Internal Event Name','Previous Event ID']] == 0).sum())


Fiscal Year                             0
Event ID                                0
Security Category Name                  0
Public Event Name                       0
Event Created Date                      0
Event Date                              0
Event Goal($)                          23
Active Registrations                    8
Inactive Registrations                122
Total Fees Paid                        56
Total of All Confirmed Gifts($)        33
Total Online Gifts($)                  45
Teams                                  35
Captains                               35
Average Team Size                      35
Team pages with custom URL            196
Emails Sent                            52
Personal pages with custom URL        158
Total Offline Confirmed Gifts($)       49
Self Donors                            49
Total From Participant($)              49
Non-self Donors                         7
Total Not From Participant($)          40
Total Team Gifts($)               

In [84]:
# Counting missing values
eventsnull=(events[['Fiscal Year','Event ID','Security Category Name','Public Event Name','Event Created Date','Event Date','Event Goal($)','Active Registrations','Inactive Registrations','Total Fees Paid','Total of All Confirmed Gifts($)','Total Online Gifts($)','Teams','Captains','Average Team Size','Team pages with custom URL','Emails Sent','Personal pages with custom URL','Total Offline Confirmed Gifts($)','Self Donors','Total From Participant($)','Non-self Donors','Total Not From Participant($)','Total Team Gifts($)','Total Event Gifts($)','Total Offline Unconfirmed Gifts($)','Street Address','City','State','Zip Code','Internal Event Name','Previous Event ID']])

print(eventsnull.isnull().sum())

# event date should have no null values, missing event dates were found to have not taken place and were removed. 102 events were originally found with null entries


Fiscal Year                             0
Event ID                                0
Security Category Name                  0
Public Event Name                       0
Event Created Date                      0
Event Date                              0
Event Goal($)                           0
Active Registrations                    0
Inactive Registrations                  0
Total Fees Paid                         0
Total of All Confirmed Gifts($)         0
Total Online Gifts($)                   0
Teams                                   0
Captains                                0
Average Team Size                       0
Team pages with custom URL              0
Emails Sent                             0
Personal pages with custom URL          0
Total Offline Confirmed Gifts($)        0
Self Donors                             0
Total From Participant($)               0
Non-self Donors                         0
Total Not From Participant($)           0
Total Team Gifts($)               

In [59]:
#compute count of values in columns
events['Public Event Name'].value_counts()

# No events should happen twice and investigations of raw data shows dupilication were "0" entries and therefore removed. 
#Additionally there were some events with data and duplicate names, they were left as is since there is no way to know if one included the values of the other
#Additionally this investigation revealed numerous other events where the sum of the following variables was zero. They were also eliminated

###########Active Registrations, 
###########Inactive Registrations, 
###########Total Fees Paid
###########Total of All Confirmed Gifts($)
###########Total Online Gifts($)
###########Teams
###########Captains
###########Average Team Size
###########Team pages with custom URL
###########Emails Sent
###########Personal pages with custom URL
###########Total Offline Confirmed Gifts($)
###########Self Donors
###########Total From Participant($)
###########Non-self Donors
###########Total Not From Participant($)
###########Total Team Gifts($)
###########Total Event Gifts($)

# the two events with duplicate entries were "Transportation", and "Bike MS: Cape Cod Getaway 2015"

Bike MS: Cape Cod Getaway 2015                                         2
Transportation                                                         2
Bike MS: Flint Hills Ride 2017                                         1
Bike MS: Deception Pass Classic 2014                                   1
Bike MS: MS 150 Ride 2017                                              1
Bike MS: Traffic Free Ride 2016                                        1
2015 Bike MS: Ride to the Rocket                                       1
Coast the Coast Bike MS 2013-old                                       1
California, San Diego - Bike MS: Bay to Bay Tour 2013                  1
Oklahoma, Chandler - Bike MS: The Mother Road Ride 2013                1
Bike MS: Valero Ride the Rim                                           1
Bike MS: Barrels and Brews 2015                                        1
2013 Bike MS: Ride the Riverside                                       1
Bike MS: Sanford Health Ride the Wind 2014         

In [85]:
#compute count of values in columns
events['City'].value_counts()

#Originally found 4 errors as follows:
#[BIKE SITE ADDRESS]	[BIKE SITE CITY]
#[BIKE SITE ADDRESS]	[BIKE SITE CITY]
#[LOCATION INFORMATION: Address]	[LOCATION INFORMATION: City]
#[BIKE SITE ADDRESS]	[BIKE SITE CITY]
#[START ADDRESS]	[START CITY]

#The entries were deleted and street address and zip code were investigated as a result


##Originally found errors as city was entered inconsistently, all inconsistencies were standardized in raw data.





Houston                                           9
West Long Branch                                  6
Easton                                            6
Pewaukee                                          6
Westminster                                       6
Miami                                             6
Biddeford                                         6
Clemmons                                          6
Hammond                                           6
Logan                                             5
Boston                                            5
Frankenmuth                                       5
Morristown                                        5
Brighton                                          5
Stratham                                          5
Orange Beach                                      5
Monmouth                                          5
Mount Vernon                                      5
North Little Rock                                 5
New Bern    

In [61]:
#compute count of values in columns
events['Street Address'].value_counts()

#Originally found errors as addresses were entered inconsistently, all inconsistencies were standardized in raw data.




10028 Ocean Gateway                                6
4061 Clemmons Road                                 6
11 Hills Beach Road                                6
800 Main Street                                    6
345 N Monmouth Avenue                              5
270 Portsmouth Avenue                              5
Log Cabin Inn & Pavilion #1                        5
3645 W 112th Ave                                   4
4701 Northshore Drive                              4
500 West 500 South                                 4
605 County Road 39                                 4
5 Waterside Crossing                               4
1523 W. Lincoln Highway                            4
1200 East 151st Street                             4
166 East 13th Street                               4
100 Edgartown - Vineyard Haven Road                4
400 Cedar Avenue                                   4
1000 Galvin Road South                             4
305 Walden Street                             

In [86]:
#compute count of values in columns
events['Zip Code'].value_counts()

#Originally found 4 errors as follows:
#[BIKE SITE ADDRESS]	[BIKE SITE CITY]
#[BIKE SITE ADDRESS]	[BIKE SITE CITY]
#[LOCATION INFORMATION: Address]	[LOCATION INFORMATION: City]
#[BIKE SITE ADDRESS]	[BIKE SITE CITY]
#[START ADDRESS]	[START CITY]

#The entries were deleted and address and zip were investigated as a result

77054.0    9
80031.0    6
27012.0    6
4005.0     6
21601.0    6
53072.0    6
7764.0     6
60115.0    5
28560.0    5
6095.0     5
1742.0     5
2125.0     5
2575.0     5
16217.0    5
44212.0    5
36561.0    5
98273.0    5
3885.0     5
84321.0    5
72118.0    5
99587.0    5
97361.0    5
43551.0    4
48116.0    4
7960.0     4
19904.0    4
48734.0    4
14478.0    4
33199.0    4
62035.0    4
          ..
83810.0    1
75024.0    1
14127.0    1
23060.0    1
30263.0    1
35756.0    1
91103.0    1
8551.0     1
67010.0    1
77210.0    1
56308.0    1
16037.0    1
28468.0    1
55805.0    1
67950.0    1
94928.0    1
39157.0    1
56560.0    1
99501.0    1
57101.0    1
5403.0     1
96817.0    1
48451.0    1
55811.0    1
10038.0    1
14456.0    1
50015.0    1
7962.0     1
20132.0    1
50403.0    1
Name: Zip Code, Length: 189, dtype: int64

In [87]:
#describe zip code for errors
events['Zip Code'].describe()

# contained 1 entry of 99999 and 1 entry of 0, and both were associated with the City errors above. they were converted to missing values


count      445.000000
mean     43510.669663
std      29548.104047
min       1237.000000
25%      16051.000000
50%      43017.000000
75%      67950.000000
max      99701.000000
Name: Zip Code, dtype: float64

# Describing Data

In [88]:
events.describe()

,Fiscal Year,Event ID,Event Goal($),Active Registrations,Inactive Registrations,Total Fees Paid,Total of All Confirmed Gifts($),Total Online Gifts($),Teams,Captains,...,Total Offline Confirmed Gifts($),Self Donors,Total From Participant($),Non-self Donors,Total Not From Participant($),Total Team Gifts($),Total Event Gifts($),Total Offline Unconfirmed Gifts($),Zip Code,Previous Event ID
count,524.000000,524.000000,5.240000e+02,524.000000,524.000000,5.240000e+02,5.240000e+02,5.240000e+02,524.000000,524.000000,...,5.240000e+02,524.000000,5.240000e+02,524.000000,5.240000e+02,524.000000,524.000000,5.240000e+02,445.000000,524.000000
mean,2014.811069,24519.307252,8.591983e+05,904.786260,34.221374,3.521103e+04,6.645060e+05,4.584677e+05,66.141221,73.406489,...,2.060382e+05,433.759542,1.199323e+05,505.248092,5.839333e+05,15031.925763,23560.687557,3.139093e+04,43510.669663,20633.131679
std,1.407638,3028.272539,1.992613e+06,1670.583736,96.799613,1.334289e+05,1.691521e+06,1.254304e+06,95.743059,109.651093,...,4.445052e+05,862.102488,3.381993e+05,914.633657,1.493962e+06,33872.280048,67993.271418,1.448310e+05,29548.104047,6962.838080
min,2013.000000,17966.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,1237.000000,0.000000
25%,2014.000000,21943.500000,1.150000e+05,162.500000,1.000000,3.139480e+03,5.831193e+04,3.525310e+04,16.000000,16.000000,...,1.746425e+04,57.000000,1.024900e+04,88.000000,4.772435e+04,642.500000,110.007500,2.437500e+02,16051.000000,18539.750000
50%,2015.000000,24822.000000,2.950000e+05,378.500000,5.500000,7.843750e+03,1.841446e+05,1.108932e+05,35.000000,36.000000,...,6.784238e+04,157.500000,3.490526e+04,221.500000,1.500580e+05,3649.720000,2802.500000,3.142500e+03,43017.000000,22041.000000
75%,2016.000000,27172.250000,9.361250e+05,1065.000000,21.250000,2.718072e+04,6.819044e+05,4.405002e+05,83.250000,90.000000,...,2.311729e+05,454.250000,1.166659e+05,617.000000,6.094167e+05,12153.457500,17370.870000,1.646975e+04,67950.000000,25304.500000
max,2017.000000,29175.000000,2.000000e+07,14871.000000,1063.000000,1.584950e+06,1.805160e+07,1.354922e+07,716.000000,781.000000,...,4.683144e+06,7283.000000,4.286214e+06,8651.000000,1.609474e+07,317524.280000,830681.600000,2.203444e+06,99701.000000,28045.000000


In [92]:
# counting 1 values

print((events[['Fiscal Year','Event ID','Security Category Name','Public Event Name','Event Created Date','Event Date','Event Goal($)','Active Registrations','Inactive Registrations','Total Fees Paid','Total of All Confirmed Gifts($)','Total Online Gifts($)','Teams','Captains','Average Team Size','Team pages with custom URL','Emails Sent','Personal pages with custom URL','Total Offline Confirmed Gifts($)','Self Donors','Total From Participant($)','Non-self Donors','Total Not From Participant($)','Total Team Gifts($)','Total Event Gifts($)','Total Offline Unconfirmed Gifts($)','Street Address','City','State','Zip Code','Internal Event Name','Previous Event ID']] == 1).sum())


Fiscal Year                            0
Event ID                               0
Security Category Name                 0
Public Event Name                      0
Event Created Date                     0
Event Date                             0
Event Goal($)                          5
Active Registrations                  18
Inactive Registrations                31
Total Fees Paid                        0
Total of All Confirmed Gifts($)        0
Total Online Gifts($)                  0
Teams                                 29
Captains                              29
Average Team Size                     19
Team pages with custom URL            43
Emails Sent                            2
Personal pages with custom URL        30
Total Offline Confirmed Gifts($)       0
Self Donors                           10
Total From Participant($)              0
Non-self Donors                       23
Total Not From Participant($)          0
Total Team Gifts($)                    0
Total Event Gift

# FIRST PRIORITY: CORPORATE ACQUISITION
  * What are the greatest growth opportunities for new corporate teams?
  * Can we apply those opportunities to specific rides/markets, especially our biggest events?
  * What industries have had the strongest involvement in Bike MS in the last five years?
  * What occupations were responsible for most of our fundraising?
  * Can we tie together these industries and occupations to identify gaps/opportunities?
  * What is the common denominator for our top performing corporate teams? (Is it industry, culture, executive involvement, connection to MS, other?)
  * Can we quantify the effect competing events are having in our top markets? 
Top competitors:
   1. Best Buddies
     * HYANNIS PORT June 2, 2018 Boston to Hyannis Port, MA
     * HEARST CASTLE September 8, 2018 San Simeon, CA
     * MIAMI November 16, 2018 Miami, FL
   2. Peddle the Cause
     * Louis
     * San Diego
   3. Pelatonia – OH
   4. PanMass Challenge - Boston
   5. ADA – Tour de Cure
   6. Jude Children’s Research Hospital
   7. JDRF Ride to Cure Diabetes
   8. Cycle for Life - Cystic Fibrosis Foundation
   9. Ride to Recovery
   10. Virtual Cycling
   11. Athletic Club Stationary Cycling Relays
   12. American Cancer Society
   13. Leukemia and Lymphoma Society – Team in Training

# SECOND PRIORITY: DIGITAL/SOCIAL ACQUISITION
  * What are the greatest opportunities for digital marketing investments? Where have we seen the greatest ROI?
  * Once someone is registered, what tactics and behaviors drive fundraising, and at what times leading up to the event?
  * What behavioral data do you see about usage of our fundraising tools and how it may or may not relate to performance of top fundraisers?
  * Despite increasing our digital advertising spend, acquisition continues to trend downward overall. Why? Is it an issue of needing more traffic, better targeting, or a conversion rate issue that needs to be addressed through the registration process? What can we do to reverse the trend?


# FINALLY:
  * As you studied this data, is there something else that came up as an insight into our operations that the questions above do not capture? 


# Descriptive Analytics